## !Important to do this first to have the effnetv2 model installed!

In [1]:
%%capture
#@title
!pip install tensorflow_addons

import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientnetv2" not in os.getcwd():
    !git clone --depth 1 https://github.com/google/automl
    os.chdir('automl/efficientnetv2')
    sys.path.append('.')
else:
    !git pull

def download(m):
    if m not in os.listdir():
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
        !tar zxf {m}.tgz
    ckpt_path = os.path.join(os.getcwd(), m)
    return ckpt_path

In [2]:
import EfficientNet_Functions as myfuncs

In [3]:
import tensorflow_hub as hub
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import csv
from PIL import Image, ImageFile
import tensorflow as tf
#import tensorflow_datasets as tfds
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Setup

In [4]:
Image.MAX_IMAGE_PIXELS = 1000000000
data_dir_train = '/projectnb/dl523/projects/Sarcasm/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full_aug/train'
data_dir_val = '/projectnb/dl523/projects/Sarcasm/wikipaintings_val'
data_dir_test = '/projectnb/dl523/projects/Sarcasm/wikipaintings_test'

#checkpoint_path = "/projectnb/dl523/students/nannkat/EC520/training/cp_unfreeze.ckpt"
model_path = '/projectnb/dl523/students/nannkat/EC520/training/effnetv2_model'

IMAGE_SIZE = 223
BATCH_SIZE =  32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

In [5]:
train_generator, valid_generator, test_generator = myfuncs.load_wikidata(data_dir_train, data_dir_val, data_dir_test,
                                                                batch_size = BATCH_SIZE, image_size = IMAGE_SIZE)


Loading data...

Found 7383 images belonging to 25 classes.
Found 246350 images belonging to 25 classes.
Found 8201 images belonging to 25 classes.
Label map:
{'Abstract_Art': 0, 'Abstract_Expressionism': 1, 'Art_Informel': 2, 'Art_Nouveau_(Modern)': 3, 'Baroque': 4, 'Color_Field_Painting': 5, 'Cubism': 6, 'Early_Renaissance': 7, 'Expressionism': 8, 'High_Renaissance': 9, 'Impressionism': 10, 'Magic_Realism': 11, 'Mannerism_(Late_Renaissance)': 12, 'Minimalism': 13, 'Naive_Art_(Primitivism)': 14, 'Neoclassicism': 15, 'Northern_Renaissance': 16, 'Pop_Art': 17, 'Post-Impressionism': 18, 'Realism': 19, 'Rococo': 20, 'Romanticism': 21, 'Surrealism': 22, 'Symbolism': 23, 'Ukiyo-e': 24}


In [6]:
import effnetv2_model

In [7]:
def unfreeze_tests(num_layers,train_generator, valid_generator, num_epochs, learning_rate):
    
    best_loss = float('inf')
    best_num = 0
    losses = []
    for i, num in enumerate(num_layers):
        
        print("Test {}/{}".format(i+1, len(num_layers)))
        print("Layer count to unfreeze: {}".format(num))
        print()
        
        tf.keras.backend.clear_session()
        model = myfuncs.get_new_model(unfreeze = num)
        checkpoint_path = "/projectnb/dl523/students/nannkat/EC520/training/cp_unfreeze" + str(num) + ".ckpt"
        model, history = myfuncs.train_effnetv2(model, train_generator, valid_generator, num_epochs = num_epochs, 
                                    learning_rate = learning_rate, restore = False,
                                   checkpoint_path = checkpoint_path)

        curr_loss = list(history['val_loss'])[-1]
        losses.append(curr_loss)
        if curr_loss < best_loss:
            best_loss = curr_loss
            best_num = num
            
        print("Loss for {} layers unfrozen: {}. Best loss is {} for {} layers unfrozen".format(num, curr_loss, 
                                                                                               best_loss, best_num))
            
            
    return losses, best_loss, best_num


In [ ]:
#80 layers: freeze powers of 2; [1,2,4,8,16,32,64,80]
#16 is 20% which gave the best result in the paper
#num epochs: try 3

num_epochs = 3
num_layers = [20, 24, 28]
losses, best_loss, best_num = unfreeze_tests(num_layers,train_generator, valid_generator, 
                                             num_epochs, LEARNING_RATE)



Test 1/3
Layer count to unfreeze: 20



Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Unfroze 20 layers
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (EffNetV2M  (None, 1280)             117746848 
 odel)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 25)                32025     
                                                                 
Total params: 117,778,873
Trainable params: 65,579,769
Non-trainable params: 52,199,104
_________________________________________________________________
Num GPUs Available:  1
Epoch 1/3
7698/7698 [==============================] - ETA: 0s - loss: 2.2597 - accuracy: 0.4053 - top_k_categorical_accuracy: 0.7827
Epoch 00001: saving model to /projectnb/dl523/students/na